In [ ]:
import sys,os
from py2neo import Graph, Node, Relationship, Database
from py2neo.matching import NodeMatcher
from py2neo.database import Schema
from Query_Parsing.CypherParser import Parse, ID_generator

In [ ]:
parent_dir = os.environ['GDMPATH']
graph = Graph("bolt:localhost:7474/databases/gdm.db", auth=("neo4j", ""))

In [ ]:
category = ['amazon']  #'dblp', 'youtube']
variant = ['small',] #['medium', 'large'] #Omitted original for now
di = {'amazon':'1', 'dblp':'2', 'youtube':'3', 'small':'4', 'medium':'5', 'large':'6'}

with open('Query_Parsing/query.json') as json_file:
    json_data = json.load(json_file)

with open('Regex_dict.json') as json_file:
    Regex_dict = json.load(json_file) 

In [ ]:
graph.delete_all()
schema = Schema(graph)
findNode = NodeMatcher(graph)

for cat in category:
    for var in variant:
        filename = "datasets/{}/{}.graph.{}".format(cat, cat, var)
        Id = ID_generators(cat,var,di)
        tx = graph.begin()
        with open(parent_dir+filename) as fp:
            elements = fp.readline().strip().split(" ")
            for i in range(0, int(elements[0])):
                tx.create(Node(ID=3000, Id.label_gen(), name=Id.name_gen(i),uid=Id.uniq_id(i),cluster=Id.gen_cluster_id(0)))
            tx.commit()
        schema.create_index(ID.label_gen(), 'uid')
        print(filename)    

In [ ]:
for cat in category:
    for var in variant:
        filename = "datasets/{}/{}.graph.{}".format(cat, cat, var)
        print(filename)
        ID = ID_generators(cat,var,di)
        tx = graph.begin()
        with open(parent_dir+filename) as fp:
            elements = fp.readline().strip().split(" ")
            for j in range(0, int(elements[1])):
                nodes = fp.readline().strip().split(" ")
                Node1 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[0]))).first()
                Node2 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[1]))).first()
                tx.create(Relationship(Node1, "KNOWS", Node2))
                tx.create(Relationship(Node2, "KNOWS", Node1))
                print("fuck you")
            tx.commit()
        print(filename)

In [ ]:
import time
start = time.time()
for cat in category:
    for var in variant:
        filename = "datasets/{}/{}.graph.{}".format(cat, cat, var)
        ID = ID_generators(cat,var,di)
#         tx = graph.begin()
        with open(parent_dir+filename) as fp:
            elements = fp.readline().strip().split(" ")
            for j in range(0, int(elements[1])):
                nodes = fp.readline().strip().split(" ")
                Node1 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[0]))).first()
                Node2 = findNode.match(ID.label_gen(), uid=ID.uniq_id(int(nodes[1]))).first()
end = time.time()
print((end-start)/60)

In [1]:
import re, json

category = ['amazon']  #'dblp', 'youtube']
variant = ['small',] #['medium', 'large'] #Omitted original for now
di = {'amazon':'1', 'dblp':'2', 'youtube':'3', 'small':'4', 'medium':'5', 'large':'6'}
   


In [11]:

 


parse = Parse(json_data, gen_dict())
Id = ID_generator('amazon', 'small', di)
print(parse.create(what='node', uid=Id.uniq_id(0), label=Id.label_gen(), name=Id.name_gen(0), cluster=Id.gen_cluster_id(0)))

CREATE (14000000:amazon_small {name=amazon_small_0, cluster=|0|})


In [12]:
print(parse.page_rank(what='query', label=Id.label_gen(), relation='KNOWS', PR_itr='20', PR_df = '0.85'))

CALL algo.pageRank.stream('amazon_small', 'KNOWS', {dampingFactor:0.85, iterations:20}) Yield nodeId, score RETURN algo.asNode(nodeId).name AS node,score ORDER BY score DESC;
